# Introduction

This notebook is written as part of the hands-on tutorial for the LLNL CCMS Summer Institute Seminar 2022 (https://pls.llnl.gov/careers/internship-programs/computational-chemistry-and-materials-science-summer-institute) given by Prof Shyue Ping Ong.

If you are running this notebook from Google Colab, uncomment the next code box to install megnet first.

In [1]:
# !pip install megnet pymatgen==2022.7.24.1

In [2]:
from megnet.utils.models import load_model, AVAILABLE_MODELS
from pymatgen.core import Structure, Lattice
from pymatgen.ext.matproj import MPRester
from pymatgen.ext.matproj import MPRestError, TaskType, _MPResterLegacy

By default, MEGNet already comes with some ready-to-use models trained on the Materials Project. 

In [3]:
print(AVAILABLE_MODELS)

['Eform_MP_2019', 'Eform_MP_2018', 'Efermi_MP_2019', 'Bandgap_classifier_MP_2018', 'Bandgap_MP_2018', 'logK_MP_2018', 'logG_MP_2018', 'logK_MP_2019', 'logG_MP_2019']


We will first use the MPRester interface to get relaxed structures from the Materials Project (https://materialsproject.org). Here, we deliberately select two very different structures:
- bcc Mo, for which MP has already computed the elastic constants (https://materialsproject.org/materials/mp-129).
- Li10GeP2S12 (LGPS), a well-known lithium superionic conductor for which MP has not calculated the elastic constants (https://materialsproject.org/materials/mp-696128).

In [4]:
# mpr = MPRester()
mpr = _MPResterLegacy('iTPrDnB1NuSywGXI')  # legacy style get the mp data

# Mo: mp-129
# Li10GeP2S12: mp-696128

structures = {}
structures["Mo"] = mpr.get_structure_by_material_id("mp-129")
structures["LGPS"] = mpr.get_structure_by_material_id("mp-696128")

/root/anaconda3/envs/megnet/lib/python3.8/site-packages/pymatgen/ext/matproj.py:183: UserWarning: You are using the legacy MPRester. This version of the MPRester will no longer be updated. To access the latest data with the new MPRester, obtain a new API key from https://materialsproject.org/api and consult the docs at https://docs.materialsproject.org/ for more information.
  warnings.warn(


In [5]:
structures

{'Mo': Structure Summary
 Lattice
     abc : 2.743237657484856 2.743237657484856 2.743237657484856
  angles : 109.47122063449069 109.47122063449069 109.47122063449069
  volume : 15.891628753050979
       A : -1.583809 1.583809 1.583809
       B : 1.583809 -1.583809 1.583809
       C : 1.583809 1.583809 -1.583809
     pbc : True True True
 PeriodicSite: Mo (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000],
 'LGPS': Structure Summary
 Lattice
     abc : 8.787646 8.787646 12.657546
  angles : 90.0 90.0 90.0
  volume : 977.4501587615295
       A : 8.787646 0.0 0.0
       B : 0.0 8.787646 0.0
       C : 0.0 0.0 12.657546
     pbc : True True True
 PeriodicSite: Li (2.0097, 2.3986, 3.7284) [0.2287, 0.2730, 0.2946]
 PeriodicSite: Li (6.7779, 6.3891, 3.7284) [0.7713, 0.7270, 0.2946]
 PeriodicSite: Li (2.3986, 6.7779, 10.0572) [0.2730, 0.7713, 0.7946]
 PeriodicSite: Li (6.3891, 2.0097, 10.0572) [0.7270, 0.2287, 0.7946]
 PeriodicSite: Li (2.0097, 6.3891, 3.7284) [0.2287, 0.7270, 0.2946]
 Periodi

In [6]:
for model_name in AVAILABLE_MODELS:
    if model_name.endswith("_2019"):
        model = load_model(model_name)
        model.metadata
        for name, structure in structures.items():
            if model_name.startswith("log"):
                prediction = 10 ** model.predict_structure(structure).ravel()[0]
            else:
                prediction = model.predict_structure(structure).ravel()[0]
            prop_name = model_name.split("_")[0].replace("log", "")
            print(f'The predicted {prop_name} for {name} is {prediction:.3f} {model.metadata["unit"].replace("log", "")}.')

2024-01-22 19:51:44.631963: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-01-22 19:51:44.645630: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-01-22 19:51:44.645719: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-01-22 19:51:44.646179: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

The predicted Eform for Mo is -0.003 eV/atom.
The predicted Eform for LGPS is -1.278 eV/atom.
The predicted Efermi for Mo is 8.401 eV.
The predicted Efermi for LGPS is 1.468 eV.
The predicted K for Mo is 268.447 10(GPa).
The predicted K for LGPS is 23.275 10(GPa).
The predicted G for Mo is 74.273 10(GPa).
The predicted G for LGPS is 9.736 10(GPa).


We may make the following observations:
- The formation energy of Mo is correctly predicted to be close to 0 (elemental reference state).
- The K of Mo is predicted to be 268 GPa, quite close to the MP value of 262 GPa. Note that this is not surprising as Mo is part of the training data for the MEGNet model to begin with. 
- In the paper "Elastic Properties of Alkali Superionic Conductor Electrolytes from First Principles Calculations" by Deng et al. [DOI: 10.1149/2.0061602jes](http://dx.doi.org/10.1149/2.0061602jes), the Materials Virtual Lab calculated the bulk modulus of LGPS to be 27.3 GPa and the shear modulus to be 7.9 GPa. This is in very good agreement with the predictions, even though it they are not in the original dataset.